<a href="https://colab.research.google.com/github/kpomal/HealthGenie/blob/main/Health_genie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install gradio langchain langchain_google_genai

In [ ]:
import gradio as gr
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from langchain_google_genai import GoogleGenerativeAI
import uuid

# Directly set your Gemini API key here
gemini_api_key = "Your API Key"  # Replace with your actual API key

# Initialize the AI model with the chosen model
llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3, api_key=gemini_api_key)

# Load the pre-trained MobileNetV2 model for image classification
image_model = tf.keras.applications.MobileNetV2(weights='imagenet')

# Function to classify image as "healthy" or "unhealthy"
def classify_image(image_file):
    # Convert the uploaded PIL image to a numpy array
    img_array = np.array(image_file)

    # Resize the image to the size required by MobileNetV2 (224x224)
    img_array = tf.image.resize(img_array, (224, 224))

    # Convert the image to NumPy array if it's a TensorFlow tensor and ensure it is writable
    img_array = img_array.numpy() if isinstance(img_array, tf.Tensor) else img_array
    img_array = img_array.copy()  # Ensure it's writable

    # Expand dimensions to match model's expected input format (batch size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)

    # Preprocess the image for MobileNetV2
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

    # Make prediction using the model
    predictions = image_model.predict(img_array)

    # Decode the top 3 predictions
    decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0]

    # Log the decoded predictions (for debugging purposes)
    print("Decoded Predictions:", decoded_predictions)

    # Classify as "healthy" or "unhealthy" based on the prediction
    healthy_keywords = ['apple', 'banana', 'carrot', 'broccoli', 'salad', 'strawberry', 'spinach', 'avocado', 'tomato', 'cucumber']
    unhealthy_keywords = ['pizza', 'burger', 'fries', 'soda', 'cake', 'chips', 'candy', 'donut', 'cookie']

    # Check if the label matches any healthy or unhealthy keywords
    for _, label, _ in decoded_predictions:
        if any(healthy_keyword in label.lower() for healthy_keyword in healthy_keywords):
            return "Healthy food!"
        elif any(unhealthy_keyword in label.lower() for unhealthy_keyword in unhealthy_keywords):
            return "Unhealthy food!"

    # If no match is found, return a default message
    return "Cannot classify food. Please try another image."

# Function to call the Gemini API for health advice
def call_gemini_api(query):
    # Few-shot examples for the model to generate health advice
    few_shot_examples = [
        "Q: What are the benefits of drinking water?\nA: - Hydration\n   - Improves skin health\n   - Aids digestion\n   - Regulates body temperature",
        "Q: How can I improve my sleep quality?\nA: - Establish a regular sleep schedule\n   - Create a relaxing bedtime routine\n   - Limit screen time before bed\n   - Keep your bedroom cool and dark",
        "Q: What are some healthy snacks?\nA: - Fresh fruits (e.g., apples, bananas)\n   - Nuts and seeds\n   - Yogurt\n   - Vegetable sticks with hummus"
    ]

    # Combine the few-shot examples with the user query to create the prompt
    prompt = "\n".join(few_shot_examples) + f"\nQ: {query}\nA:"

    # Generate response from the AI model
    response = llm.generate([prompt])  # Pass the prompt as a single string
    response_text = response.generations[0][0].text.strip()

    # Format the response into bullet points for better readability
    formatted_response = "\n".join([f"- **{line.strip()}**" for line in response_text.split("\n") if line.strip()])

    # Generate a unique filename for the response
    file_name = f"response_{uuid.uuid4().hex}.txt"

    # Save the response to a text file for download
    with open(file_name, "w") as f:
        f.write(formatted_response)

    return formatted_response, file_name  # Return the formatted response and the file path

# Create the Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("# HealthGenie AI Assistant")

    with gr.Row():
        with gr.Column():
            # Text-based AI Assistant for health queries
            query_input = gr.Textbox(label="Ask HealthGenie anything:", placeholder="Type your question here...")
            response_output = gr.Textbox(label="Response", interactive=False)
            download_file = gr.File(label="Download Response")
            query_input.submit(call_gemini_api, query_input, [response_output, download_file])

        with gr.Column():
            # Image classification section
            image_input = gr.Image(type="numpy", label="Upload an image of food")
            classify_button = gr.Button("Classify Image")
            output_label = gr.Textbox(label="Food Classification", interactive=False)

            # When the classify button is clicked, the image classification function is called
            classify_button.click(classify_image, image_input, output_label)

# Launch the Gradio app
iface.launch(share=True)  # Enable sharing to generate a URL to interact with the interface
